In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def bayesian_ridge_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_iter=300,
    alpha_1=1e-6,
    alpha_2=1e-6,
    lambda_1=1e-6,
    lambda_2=1e-6,
    compute_scores=True,
    random_state=None
):
    """
    Bayesian Ridge Regression for time series forecasting with:
    - Probabilistic uncertainty estimates
    - Automatic complexity control
    - Lag features only
    """
    # Create directory structure
    os.makedirs('BayesianRidgeRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 3. Train-Test Split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model Training with Bayesian Ridge
    model = BayesianRidge(
        n_iter=n_iter,
        alpha_1=alpha_1,
        alpha_2=alpha_2,
        lambda_1=lambda_1,
        lambda_2=lambda_2,
        compute_score=compute_scores,
    )
    model.fit(X_train, y_train)

    # 5. Forecasting with Uncertainty
    pred_test, pred_test_std = model.predict(X_test, return_std=True)

    # 6. Inverse Differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Create forecast DataFrame with uncertainty
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast_mean': pred_test,
        'forecast_std': pred_test_std
    })

    # 8. Save forecasts
    forecast_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_forecasts.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, mode='a', header=write_header, index=False)

    # 9. Calculate metrics and uncertainty statistics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast_mean']))
    log_marginal_likelihood = model.scores_[-1] if compute_scores else np.nan
    
    # Calculate coefficient statistics
    coef_means = model.coef_
    coef_stds = np.sqrt(np.diag(model.sigma_))
    
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'log_marginal_likelihood': log_marginal_likelihood,
        'coefficient_means': coef_means.tolist(),
        'coefficient_stds': coef_stds.tolist(),
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan,
        'effective_params': model.lambda_ / model.alpha_,
        'differencing': d,
        'n_iter': n_iter
    }])

    metrics_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_metrics.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, mode='a', header=write_header_metrics, index=False)

    # 10. Plotting with uncertainty bands
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast_mean'], label='Mean Forecast', color='darkorange')
    plt.fill_between(
        forecast_df['date'],
        forecast_df['forecast_mean'] - 1.96*forecast_df['forecast_std'],
        forecast_df['forecast_mean'] + 1.96*forecast_df['forecast_std'],
        color='orange',
        alpha=0.2,
        label='95% Confidence'
    )
    plt.title(
        f'Bayesian Ridge Forecast for {district_name}\n'
        f'RMSE: {rmse:.2f} | Log ML: {log_marginal_likelihood:.2f}'
    )
    plt.legend()
    plt.savefig(os.path.join('BayesianRidgeRegression', f'bayesian_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model': model,
        'coef_means': coef_means,
        'coef_stds': coef_stds,
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        results = bayesian_ridge_regression_lags_only(
            ts,
            district,
            max_lags=4,
            n_iter=500,
            alpha_1=1e-5,
            lambda_1=1e-3
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
     district       rmse  log_marginal_likelihood  \
0  AHMEDNAGAR  93.433481              -207.222332   

                                   coefficient_means  \
0  [0.30175998094197815, 0.0501714889804589, 0.01...   

                                    coefficient_stds  intercept_mean  \
0  [0.1302223713219305, 0.13812698552307828, 0.13...      323.131363   

   intercept_std  effective_params  differencing  n_iter  
0            NaN       172216.1789             0     500  

=== Forecast Data ===
              district       date  actual  forecast_mean  forecast_std
date                                                                  
2020-07-01  AHMEDNAGAR 2020-07-01   354.0     428.554607    117.226908
2020-08-01  AHMEDNAGAR 2020-08-01   391.0     433.840474    109.338554
2020-09-01  AHMEDNAGAR 2020-09-01   417.0     448.912628    110.537414
2020-10-01  AHMEDNAGAR 2020-10-01   343.0     453.343328    114.162341
2020-11-01  AHMEDNAGAR 2020-11-01   468.0     434.41

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
  district       rmse  log_marginal_likelihood  \
0    AKOLA  67.458159              -206.961811   

                                   coefficient_means  \
0  [0.0007478335897655638, -0.0001493157008442984...   

                                    coefficient_stds  intercept_mean  \
0  [0.011971959405631854, 0.0119719396468928, 0.0...      449.138516   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      5.560266e+07             0     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01    AKOLA 2020-07-01   407.0     449.347535     90.041171
2020-08-01    AKOLA 2020-08-01   306.0     449.332832     90.068204
2020-09-01    AKOLA 2020-09-01   504.0     449.461275     90.042538
2020-10-01    AKOLA 2020-10-01   493.0     449.470868     89.997805
2020-11-01    AKOLA 2020-11-01   514.0     449.517959     90.099666


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


...

=== Metrics ===
  district        rmse  log_marginal_likelihood  \
0    THANE  262.598543              -225.106069   

                                   coefficient_means  \
0  [-0.0005617450201543172, -0.000539312814121165...   

                                    coefficient_stds  intercept_mean  \
0  [0.011420716993542959, 0.011420530943540367, 0...      -18.262687   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      2.516196e+08             1     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01    THANE 2020-07-01     708     730.760398    181.607341
2020-08-01    THANE 2020-08-01     893     712.615278    181.607479
2020-09-01    THANE 2020-09-01     916     694.176718    181.614203
2020-10-01    THANE 2020-10-01    1156     675.782204    181.614391
2020-11-01    THANE 2020-11-01     848     657.476755    181.625002
=== Me

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
  district      rmse  log_marginal_likelihood  \
0   WASHIM  21.85879              -181.573953   

                                   coefficient_means  \
0  [0.43539352167465833, 0.02975752096149019, 0.0...   

                                    coefficient_stds  intercept_mean  \
0  [0.13962857765432507, 0.13045321800971696, 0.1...       70.978609   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      22513.085399             0     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01   WASHIM 2020-07-01   154.0     117.905344     44.072922
2020-08-01   WASHIM 2020-08-01   127.0     140.659274     45.718824
2020-09-01   WASHIM 2020-09-01   120.0     125.696619     45.844661
2020-10-01   WASHIM 2020-10-01   113.0     128.253074     45.642519
2020-11-01   WASHIM 2020-11-01   154.0     116.796895     45.870437
=== Metric

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


{'district': 'MUMBAI SUBURBAN',
 'forecast_df':                    district       date  actual  forecast_mean  forecast_std
 date                                                                       
 2021-02-01  MUMBAI SUBURBAN 2021-02-01   607.0     692.322686    558.723456
 2021-03-01  MUMBAI SUBURBAN 2021-03-01   618.0     776.574762    498.382201,
 'metrics_df':           district        rmse  log_marginal_likelihood  \
 0  MUMBAI SUBURBAN  127.330114               -38.369483   
 
                                    coefficient_means  \
 0  [0.04145388365613513, -0.10246771260009085, -0...   
 
                                     coefficient_stds  intercept_mean  \
 0  [0.26082303717895683, 0.2590727207333168, 0.26...     1417.294754   
 
    intercept_std  effective_params  differencing  n_iter  
 0            NaN      91092.815641             0     500  ,
 'model': BayesianRidge(alpha_1=1e-05, compute_score=True, lambda_1=0.001, n_iter=500),
 'coef_means': array([ 0.04145388, 